# PyCAL86:  CAL86 Work-Alike
This module implement enough of CAL86 for the CIVE 4200 assignments.  CAL commands seem to be C,
LOAD, P, PRINT, ADD, MULT, TRAN, ZERO, LOADI, FRAME, ADDK, SOLVE and  MEMFRC.  In addition, we add
the following commands:  TTMULT, PSOLVE and TRUSS2D

This module implements one %%CELL86 cell magic, which then reads and interprets
the following lines in that cell.  The CAL commands work with variables in the normal notebook
namespace and so these variables can be examined and manipulated outside of CAL, in normal
notebook cells.

#### Available commands:

* [ADD](#ADD-Command) - Form the sum of two matrices.
* [ADDK](#ADDK-Command) - Add an element stiffness to the structure stiffness.
* [C](#C-Command) - Comment your code.
* [FRAME](#FRAME-Command) - Form a frame element stiffness matrix.
* [HELP](#HELP-Command) - Help.
* [LOAD](#LOAD-Command) - Load float point numbers int a matrix.
* [LOADI](#LOADI-Command) - Load integer numbers into a matrix (GLOBAL DOF indices, usually).
* [MEMFRC](#MEMFRC-Command) - Calculate member end forces from displacements.
* [MULT](#MULT-Command) - Form the product of two matrices.
* [P](#P-Command) - Print a matrix.
* [PRINT](#PRINT-Command) - Print a matrix.
* [PSOLVE](#PSOLVE-Command) - Solve a partitioned set of equations.
* [SOLVE](#SOLVE-Command) - Solve a set of equations.
* [TMULT](#TMULT-Command) - Multiply a matrix by the transpose of a matrix.
* [TRAN](#TRAN-Command) - Form the transpose of a matrix.
* [TRUSS](#TRUSS-Command) - Form a truss element stiffness matrix.
* [TTMULT](#TTMULT-Command) - Form the product $[T]^T [K] [T]$ from $[T]$ and $[K]$.
* [ZERO](#ZERO-Command) - Create a matrix of zeros.

**Note:** An interactive version of this documentation page is available at
[http://holtz2.cee.carleton.ca:8000/notebooks/cive4200/PyCAL86-Doc.ipynb](http://holtz2.cee.carleton.ca:8000/notebooks/cive4200/PyCAL86-Doc.ipynb).
If you want to experiment with any of its interactive cells, remember to first select and excecute
menu item **Cell / Run All** from the top menu bar.


In [1]:
import PyCAL86

# Documentation and Help

## HELP Command

### Usage: HELP
The **HELP** command displays some help.

In [2]:
%%CAL86
HELP


Available CAL commands: ADD, ADDK, C, FRAME, HELP, LOAD, LOADI, MEMFRC, MULT, P, PRINT, PSOLVE, SOLVE, TMULT, TRAN, TRUSS, TTMULT, ZERO


You can also get help on an individual command:

In [3]:
%%CAL86
HELP TTMULT


TTMULT:
Multiply two matrices.
    
    Usage:
    
    TTMULT  M1  M2  M3
    
    The matrix product M1' * M2 * M1 is formed
    and stored in variable M3.


## C Command

### Usage: C
The **C** command is a comment and is ignored during processing.

In [4]:
%%CAL86
C this is a comment
C and so is this
C me too

# Matrix Commands

## PRINT Command
## P Command

### Usage: PRINT $M$ (or P $M$)
The **PRINT** or **P** command will cause the matrix in variable $M$ to be displayed on the notebook.

In [5]:
%%CAL86
ZERO c R=3 C=3 D=3
print c


c:
[[ 3.  0.  0.]
 [ 0.  3.  0.]
 [ 0.  0.  3.]]


## LOAD Command

### Usage: LOAD   $M$   R=$nr$   C=$nc$
The **LOAD** command will create a matrix in variable $M$, with $nr$ rows and $nc$ columns.
The data must immediately follow the LOAD command and it must be supplied one row per line.
The column values must be separated by one comma, and/or one or more spaces.  A line of 
data may be continued by use of a '\' character at the end of each continued line.

In [6]:
%%CAL86
LOAD   xx     R=2    C=5
1 2 3 \
  4 5
10 20 30 40 50
PRINT xx


xx:
[[  1.   2.   3.   4.   5.]
 [ 10.  20.  30.  40.  50.]]


CAL matrices are just variables in the interactive notebook workspace,
so that they can be directly used in IPython interactions:

In [7]:
xx*xx.T

matrix([[   55.,   550.],
        [  550.,  5500.]])

Expressions containing notebook variables may be used as elements in the
matrix data, as long as there are no spaces in the expression.  This allows for
some convenience in building matrices.  For example:

In [8]:
E = 200000.
A = 1000.
L = 5000.
k = E*A/L

In [9]:
%%CAL86
LOAD LK R=4 C=4
k 0 -k 0
0 0 0 0
-k 0 2*k 0
0 0 0 0
PRINT LK


LK:
[[ 40000.      0. -40000.      0.]
 [     0.      0.      0.      0.]
 [-40000.      0.  80000.      0.]
 [     0.      0.      0.      0.]]


## LOADI Command

### Usage: LOADI  $M$   R=$nr$  C=$nc$
The **LOADI** command will create an integer matrix in variable $M$, with $nr$ rows and $nc$ columns.
The data must immediately follow the LOAD command and it must be supplied one row per line.
The column values must be separated by one comma, and/or one or more spaces.  A line of 
data may be continued by use of a '\' character at the end of each continued line.
All data values are converted to integers.

See the [ADDK Command](#ADDK-Command), below for the normal use of LOADI and for the meaning of the data.

In [10]:
%%CAL86
LOADI mm R=3 C=4
1 2                 3               4.999
5 6 7 8+5
9 10 11 \
12
PRINT mm


mm:
[[ 1  2  3  4]
 [ 5  6  7 13]
 [ 9 10 11 12]]


In [11]:
mm.dtype

dtype('int64')

## TRAN Command

### Usage: TRAN  $M_1$  $M_2$
The **TRAN** command forms the transpose of the matrix in $M_1$ and places it in $M_2$.

In [12]:
%%CAL86
TRAN xx yy
PRINT xx
PRINT yy


xx:
[[  1.   2.   3.   4.   5.]
 [ 10.  20.  30.  40.  50.]]
yy:
[[  1.  10.]
 [  2.  20.]
 [  3.  30.]
 [  4.  40.]
 [  5.  50.]]


## ADD Command

### Usage: ADD  $M_1$  $M_2$
The **ADD** command adds the matrices in $M_1$ and $M_2$ and places
the result back in $M_1$, modifying it in the process.   (Editors note:  this is highly inconsistent!! WHY is it done this way????
No other commands do this type of thing.  Sheesh!)

In [13]:
%%CAL86
PRINT xx
ADD xx xx
PRINT xx


xx:
[[  1.   2.   3.   4.   5.]
 [ 10.  20.  30.  40.  50.]]
xx:
[[   2.    4.    6.    8.   10.]
 [  20.   40.   60.   80.  100.]]


## MULT Command

### Usage: MULT  $M_1$  $M_2$  $M_3$
The **MULT** command multiplies the matrices in $M_1$ and $M_2$ and places the result in $M_3$.

In [14]:
%%CAL86
MULT xx yy zz
PRINT xx
PRINT yy
PRINT zz


xx:
[[   2.    4.    6.    8.   10.]
 [  20.   40.   60.   80.  100.]]
yy:
[[  1.  10.]
 [  2.  20.]
 [  3.  30.]
 [  4.  40.]
 [  5.  50.]]
zz:
[[   110.   1100.]
 [  1100.  11000.]]


## TMULT Command

### Usage: TMULT  $M_1$  $M_2$  $M_3$
The **TMULT** command multiplies the the transpose of the  matrix in $M_1$ and the matrix in $M_2$ and places the result in $M_3$.

In [15]:
%%CAL86
LOAD a R=2 C=3
1 2 3
1 2 3
LOAD b R=2 C=3
10 20 30
10 20 30
TMULT a b qq
PRINT a
PRINT b
PRINT qq


a:
[[ 1.  2.  3.]
 [ 1.  2.  3.]]
b:
[[ 10.  20.  30.]
 [ 10.  20.  30.]]
qq:
[[  20.   40.   60.]
 [  40.   80.  120.]
 [  60.  120.  180.]]


## TTMULT Command

### Usage: TTMULT  $M_1$  $M_2$  $M_3$
The **TTMULT** command forms the product $M_1^T \times M_2 \times M_1$ and places the result in $M_3$.

In [16]:
%%CAL86
LOAD A R=2 C=2
1 2
3 4
PRINT A
LOAD B R=2 C=2
10 20
30 40
PRINT B
TTMULT A B C
PRINT C
C The TMULT command replaces these steps:
TRAN A AT
MULT AT B x
MULT x A D
PRINT D


A:
[[ 1.  2.]
 [ 3.  4.]]
B:
[[ 10.  20.]
 [ 30.  40.]]
C:
[[  520.   760.]
 [  740.  1080.]]
D:
[[  520.   760.]
 [  740.  1080.]]


## ZERO Command

### Usage: ZERO  $M_1$  R=$nr$  C=$nc$  [T=$t$]  [D=$d$]
The **ZERO** command creates a matrix of size $nr\times nc$.  If $t$ is specified, all elements
will be set to this value (the default value of $t$ is 0 (zero)).  If $d$ is specified and the
matrix is square ($nr=nc$), the diagonal values will be set to this (the default value of $d$
is $t$).

In [17]:
%%CAL86
ZERO QQ R=5 C=5  T=3  D=7
PRINT QQ
ZERO QR R=3 C=4
PRINT QR


QQ:
[[ 7.  3.  3.  3.  3.]
 [ 3.  7.  3.  3.  3.]
 [ 3.  3.  7.  3.  3.]
 [ 3.  3.  3.  7.  3.]
 [ 3.  3.  3.  3.  7.]]
QR:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


# Structures Commands

## FRAME Command

### Usage: FRAME  $GK$ $FD$  I=$I_x$  A=$A$  E=$E$  X=$x_j,x_k$  Y=$y_j,y_k$
The **FRAME** command forms the 6x6 global element stiffness matrix, $GK$, and a 4x6 force-displacement
matrix $FD$ for a general two-dimensional bending member with axial deformations included in
the formulation.  The properties of the member are given as:
  > $I_x$ = the moment of inertia of the member, and
  
  > $A$ = the cross-sectional area of the member, and
  
  > $E$ = the Modulus of Elasticity of the member.
  
The coordinates of the "$j$" and "$k$" ends of the member are defined by $x_j,x_k$ and
$y_j,y_k$ respectively.  Note that the user is responsible for the definition of the $j$ and
$k$ ends of the member.

This command computes the local element stiff matrix, $[LK]$, and the geometric transformation matrix $[T]$.  
It forms $[GK] = [T]^T \times [LK] \times[T]$ and $[FD] = [LK] \times [T]$.  The $[T]$ matrix is not stored.

![signs](img/2d-signs-v2.svg)

In [18]:
%%CAL86
FRAME K4 T4 I=1000 A=20 E=30000   X=400,350 Y=0,150
PRINT K4
PRINT T4


K4:
[[    461.43956   -1111.09788   -6830.51975    -461.43956    1111.09788   -6830.51975]
 [  -1111.09788    3424.36723   -2276.83992    1111.09788   -3424.36723   -2276.83992]
 [  -6830.51975   -2276.83992  758946.63844    6830.51975    2276.83992  379473.31922]
 [   -461.43956    1111.09788    6830.51975     461.43956   -1111.09788    6830.51975]
 [   1111.09788   -3424.36723    2276.83992   -1111.09788    3424.36723    2276.83992]
 [  -6830.51975   -2276.83992  379473.31922    6830.51975    2276.83992  758946.63844]]
T4:
[[  -6830.51975   -2276.83992  758946.63844    6830.51975    2276.83992  379473.31922]
 [  -6830.51975   -2276.83992  379473.31922    6830.51975    2276.83992  758946.63844]
 [   1200.        -3600.            0.        -1200.         3600.            0.     ]
 [    -86.4         -28.8        7200.           86.4          28.8        7200.     ]]


## TRUSS Command

### Usage: TRUSS  $GK$ $FD$  A=$A$  E=$E$  X=$x_j,x_k$  Y=$y_j,y_k$
The **TRUSS** command forms the 4x4 element stiffness matrix, $GK$, and a 4x4 force-displacement
matrix $FD$ for a general two-dimensional truss member with only axial deformations included in
the formulation.  The properties of the member are given as

  > $A$ = the cross-sectional area of the member, and
  
  > $E$ = the Modulus of Elasticity of the member.
  
The coordinates of the "$j$" and "$k$" ends of the member are defined by $x_j,x_k$ and
$y_j,y_k$ respectively.  Note that the user is responsible for the definition of the $j$ and
$k$ ends of the member.

This command computes the local element stiff matrix, $[LK]$, and the geometric transformation matrix $[T]$.  
It forms $[GK] = [T]^T \times [LK] \times[T]$ and $[FD] = [LK] \times [T]$.  The $[T]$ matrix is not stored.

![signs](img/2dtruss-signs-v2.svg)

In [19]:
%%CAL86
TRUSS K5 T5 A=1000 E=200000 X=1000,5000 Y=2000,5000
PRINT K5
PRINT T5


K5:
[[ 25600.  19200. -25600. -19200.]
 [ 19200.  14400. -19200. -14400.]
 [-25600. -19200.  25600.  19200.]
 [-19200. -14400.  19200.  14400.]]
T5:
[[ 32000.  24000. -32000. -24000.]
 [     0.      0.      0.      0.]
 [-32000. -24000.  32000.  24000.]
 [     0.      0.      0.      0.]]


## ADDK Command

### Usage: ADDK  $K$  $GK$  $ID$  N=$n$
The global element stiffness matrix $GK$ is added to the total stiffness matrix $K$.  The row and column numbers where the terms are to be added are obtained from column $n$ of the $L\times m$ integer matrix $ID$ (where $m$ is the total number of members and $L$ is the size of $EK$ - either 4x4 or 6x6).

The $ID$ matrix gives the correspondence between the global DOF#s and the local DOF#s for each member.  There
is one column for each member, column $k$ corresponds to member $k$.  The numbers from top to bottom are the
global DOF#s in the same order as the local DOF#s. EG: the top number is the global DOF# corresponding to local
DOF #0 of the member, the next down corresponds to local DOF #1, etc.

![signs](img/2d-signs-v2-eg.svg)

When determing the mapping between local numbers, it is useful to think of the element in its "normal" unrotated
position, with its "j" end at the start node.
For example, consider the 2D frame element, above.  If it is defined as going from node *a* to *b*, then both axes projections
will be positive, as will $\cos\theta_x$ and $\cos\theta_y$.  In this case, the "j" end will be
at node *a* and the local numbers 0, 1 and 2 at the "j" end
will map to the global numbers 6, 4 and 2 at node a; the full column for this member will contain 6,4,2,12,8,10.

If the element is considered as going from node *b* to *a*, then both projections and cosines will be
negative.  End "j" of the unrotated element will correspond to node *b* and the local numbers 0, 1 and 2 there
will correspond to global numbers 12,8, and 10.  The full column of $ID$ will be 12,8,10,6,4,2.

In [20]:
%%CAL86
LOAD K1 R=6 C=6
100 101 102 103 104 105
110 111 112 113 114 115
120 121 122 123 124 125
130 131 132 133 134 135
140 141 142 143 144 145
150 151 152 153 154 155
LOADI IN R=6 C=4
0 1 2 3
1 2 3 4
2 3 4 5
3 7 5 6
4 8 6 7
5 9 7 8
PRINT K1
PRINT IN
ZERO K R=10 C=10
ADDK K K1 IN N=1
PRINT K

ADDK K K1 IN N=1
PRINT K


K1:
[[ 100.  101.  102.  103.  104.  105.]
 [ 110.  111.  112.  113.  114.  115.]
 [ 120.  121.  122.  123.  124.  125.]
 [ 130.  131.  132.  133.  134.  135.]
 [ 140.  141.  142.  143.  144.  145.]
 [ 150.  151.  152.  153.  154.  155.]]
IN:
[[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]
 [3 7 5 6]
 [4 8 6 7]
 [5 9 7 8]]
K:
[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  100.  101.  102.    0.    0.    0.  103.  104.  105.]
 [   0.  110.  111.  112.    0.    0.    0.  113.  114.  115.]
 [   0.  120.  121.  122.    0.    0.    0.  123.  124.  125.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  130.  131.  132.    0.    0.    0.  133.  134.  135.]
 [   0.  140.  141.  142.    0.    0.    0.  143.  144.  145.]
 [   0.  150.  151.  152.    0.    0.    0.  153.  154.  155.]]
K:
[[   0.    0.    0.    0.    0.    0.    0.    0.   

## PSOLVE Command

### Usage: PSOLVE  $A$  $D$  $P$  PS=$p$
The command **PSOLVE** solves the linear set of equations $A D = P$ and places the result back into matrices $D$ and $P$.
If PS=$p$ is given, then $p$ is the unconstrained partition size (the number of unconstrained degrees
of freedom), and the partion $A_{uu} D_u + A_{uc} D_c = P_u$ is solved for $D_u$ with given $D_c$ (support
displacements - normally 0).
For a partioned solution, $P_c$ (the support reactions) are determined from: $P_c = A_{cu} D_u + A_{cc} D_c$.

First, just create some artificial K, D and P matrices such that K D = P.

In [21]:
%%CAL86
LOAD K R=5 C=5
5 3 2 1 0
3 4 3 2 1
2 3 5 3 1
1 2 3 6 2
0 1 1 2 5

LOAD D R=5 C=1
10
20
30
20
10

MULT K D P
PRINT D
PRINT P


D:
[[ 10.]
 [ 20.]
 [ 30.]
 [ 20.]
 [ 10.]]
P:
[[ 190.]
 [ 250.]
 [ 300.]
 [ 280.]
 [ 140.]]


The partition size will be 3 (of 5 total).  Set the forces at the contrained DOFS to zero and the displacements
and the unconstrained DOFS to zero.  These will then be determined by PSOLVE.

In [22]:
P[3:,:] = 0     # set forces at constrained DOF to 0.  So we can see what happens when computed later.
D[:3,:] = 0     # set displacements at unconstrained DOF to 0.  So we can see what happens when computed later.

Solve and print the total D and P matrices.

In [23]:
%%CAL86
PRINT D
PRINT P

PSOLVE K D P PS=3
PRINT D
PRINT P


D:
[[  0.]
 [  0.]
 [  0.]
 [ 20.]
 [ 10.]]
P:
[[ 190.]
 [ 250.]
 [ 300.]
 [   0.]
 [   0.]]
D:
[[ 10.]
 [ 20.]
 [ 30.]
 [ 20.]
 [ 10.]]
P:
[[ 190.]
 [ 250.]
 [ 300.]
 [ 280.]
 [ 140.]]


Try a full (unpartitioned) solve and see that *x* is the full set of displacements and *P* is unmodified.

In [24]:
%%CAL86
PSOLVE K x P
PRINT x
PRINT P


x:
[[ 10.]
 [ 20.]
 [ 30.]
 [ 20.]
 [ 10.]]
P:
[[ 190.]
 [ 250.]
 [ 300.]
 [ 280.]
 [ 140.]]


## SOLVE Command

### Usage: SOLVE  $A$  $B$  S=$p$
The command **SOLVE** solves the linear set of equations $A x = B$ for $x$ and places the result $x$ in matrix $B$.
If S=$p$ is given, then $p$ may only have the value 0.

This command is only for compatibilty with traditional CAL.  It will rarely be used.

## MEMFRC Command

### Usage: MEMFRC  $FD$  $U$  $ID$  $P$  N=$n$
The member forces are evaluated by multiplying the force-displacement matrix $FD$ by the node displacements in $U$ and
storing the results in matrix $P$.  The force displacement matrix $FD$ is that computed by the [FRAME](#FRAME-Command)
and [TRUSS](#TRUSS-Command) commands.

The node displacements that are used are obtained using the global DOF#s in
column $n$ of integer array $ID$.  If $FD$ is the 6x6 global element stiffness matrix returned as $GK$ as the first
matrix of the FRAME command, the forces are given according to the global coordinate system.
If $FD$ is the 4x6 force-displacement transformation matrix returned as $FD$ as the second matrix of the FRAME command, 
the forces will be given in a simplified, 4-element local coordinate system.

In [25]:
%%CAL86
FRAME K1 T1 I=1000 A=20 E=30000 X=0,0 Y=100,0

LOADI IN R=6 C=3
0 3 0
1 4 1
2 5 2
3 6 6
4 7 7
5 8 8

LOAD D R=10 C=1
1
2
3
4
5
6
7
8
9
10

PRINT K1
PRINT T1
PRINT IN
PRINT D

MEMFRC K1 D IN P1 N=0
PRINT P1

MEMFRC T1 D IN P2 N=2
PRINT P2


K1:
[[     360.        0.    18000.     -360.        0.    18000.]
 [       0.     6000.        0.        0.    -6000.        0.]
 [   18000.        0.  1200000.   -18000.        0.   600000.]
 [    -360.        0.   -18000.      360.        0.   -18000.]
 [       0.    -6000.        0.        0.     6000.        0.]
 [   18000.        0.   600000.   -18000.        0.  1200000.]]
T1:
[[   18000.        0.  1200000.   -18000.        0.   600000.]
 [   18000.        0.   600000.   -18000.        0.  1200000.]
 [       0.     6000.        0.        0.    -6000.        0.]
 [     360.        0.    18000.     -360.        0.    18000.]]
IN:
[[0 3 0]
 [1 4 1]
 [2 5 2]
 [3 6 6]
 [4 7 7]
 [5 8 8]]
D:
[[  1.]
 [  2.]
 [  3.]
 [  4.]
 [  5.]
 [  6.]
 [  7.]
 [  8.]
 [  9.]
 [ 10.]]
P1:
[[  160920.]
 [  -18000.]
 [ 7146000.]
 [ -160920.]
 [   18000.]
 [ 8946000.]]
P2:
[[  8892000.]
 [ 12492000.]
 [   -36000.]
 [   213840.]]


D[[0,1,2,3,4,5],:]

In [26]:
K1*D[[0,1,2,3,4,5],:]

matrix([[  160920.],
        [  -18000.],
        [ 7146000.],
        [ -160920.],
        [   18000.],
        [ 8946000.]])

In [27]:
D[[0,1,2,6,7,8],:]

matrix([[ 1.],
        [ 2.],
        [ 3.],
        [ 7.],
        [ 8.],
        [ 9.]])

In [28]:
T1*D[[0,1,2,6,7,8],:]

matrix([[  8892000.],
        [ 12492000.],
        [   -36000.],
        [   213840.]])

# The above MEMFRC stuff has to be checked.  It has changed from the CAL way!!!